# 🏛️ VoxCity Landmark Visibility Analysis

Analyze **visibility** of specific landmarks (buildings) from all locations in your study area using 3D ray-casting.

## Use Cases

- **Wayfinding research** - Which landmarks are visible from different locations?
- **Urban planning** - Assess visual impact of new buildings
- **Real estate** - Evaluate views from different properties
- **Tourism** - Map visibility of attractions

## Landmark Selection Methods

| Method | When to Use |
|--------|-------------|
| `landmark_building_ids` | You know specific building IDs |
| `landmark_polygon` | Define custom area containing landmarks |
| Auto-select (center) | Let VoxCity pick buildings near center |

## Prerequisites

```python
pip install voxcity
```

In [ ]:
# %pip install voxcity

from voxcity.generator import get_voxcity
from voxcity.geoprocessor.draw import draw_rectangle_map_cityname
from voxcity.simulator.view import get_landmark_visibility_map

meshsize = 5
cityname = "Tokyo, Japan"

rectangle_vertices = [
    (139.760, 35.680),  # SW
    (139.760, 35.690),  # NW
    (139.770, 35.690),  # NE
    (139.770, 35.680)   # SE
]

city = get_voxcity(
    rectangle_vertices,
    meshsize=meshsize,
    building_source='OpenStreetMap',
    land_cover_source='OpenStreetMap',
    canopy_height_source='High Resolution 1m Global Canopy Height Maps',
    dem_source='DeltaDTM',
    output_dir='output/landmark_demo'
)

# Access building_gdf from the VoxCity object
building_gdf = city.extras.get('building_gdf', None)
len(building_gdf) if building_gdf is not None else 0


---
## 🎯 Select Landmarks

Three ways to specify which buildings are "landmarks":

1. **By IDs**: Provide `landmark_building_ids` - list of building indices from GeoDataFrame
2. **By Polygon**: Provide `landmark_polygon` - GeoJSON-style polygon, all buildings inside become landmarks
3. **Auto-center**: Omit both parameters - function picks buildings at rectangle center

In [ ]:
landmark_kwargs = {
    "view_point_height": 1.5,
    "colormap": "cool",
    "obj_export": True,
    "output_directory": "output/landmark_demo",
    "output_file_name": "landmark_visibility",
    "alpha": 1.0,
}

# Example 1: Pick by IDs (take first few as sample)
ids_sample = building_gdf.head(3).index.tolist() if building_gdf is not None else []

landmark_vis_map, voxcity_grid_marked = get_landmark_visibility_map(
    city,
    building_gdf=building_gdf,
    landmark_building_ids=ids_sample,
    **landmark_kwargs
)

landmark_vis_map.shape


In [ ]:
# Example 2: Auto-select using rectangle center (omit IDs and polygon)
landmark_vis_map_auto, _ = get_landmark_visibility_map(
    city,
    building_gdf=building_gdf,
    rectangle_vertices=rectangle_vertices,
    **landmark_kwargs
)

landmark_vis_map_auto.shape


---
## 📊 Interpreting Results

- **Visibility values**: 0 = not visible, 1 = fully visible
- **Output**: Colored OBJ mesh showing visibility from each location
- **`voxcity_grid_marked`**: 3D voxel grid with landmarks highlighted

## Next Steps

- `demo_view.ipynb` - Green View Index & Sky View Index
- `demo_solar.ipynb` - Solar irradiance simulation
- `demo_3d_visualization.ipynb` - Advanced 3D visualization